Katie Briggs
DSC550-301
Week 4 Clustering

# 1. Expectation Maximization Clustering

Write a script that implements the Expectation-Maximization (EM) algorithm for clustering (see Algorithm 13.3 in Chapter 13).
Run the code on the iris.txt dataset.
Use the first four attributes for clustering, and use the labels only for the purity-based clustering evaluation (see below). Estimate the full covariance matrix for each cluster.

For EM initialization, use the first n/k points for cluster 1, the next n/k for cluster 2, and so on. For convergence testing, you can compare the sum of the euclidean distance between the old means and the new means over the k clusters. If this distance is less than ϵ=0.001 you can stop the method.

Your program output should consist of the following information:

The final mean for each cluster
The final covariance matrix for each cluster
Number of iterations the EM algorithm took to converge.
Final cluster assignment of all the points, where each point will be assigned to the cluster that yields the highest probability P(Ci|xj)
Final size of each cluster
Finally, you must compute the 'purity score' for your clustering, computed as follows: Assume that Ci denotes the set of points assigned to cluster i by the EM algorithm, and let Ti denote the true assignments of the points based on the last attribute. Purity score is defined as:

Purity=1n∑i=1kmaxkj=1{Ci∩Tj


In [36]:
# import libraries needed for this exercise

import numpy as np
import pandas as pd
from scipy.stats import multivariate_normal as mvn
import sys
import time
from sklearn.datasets import load_iris
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt

In [37]:
# read in iris dataframe from notebook

iris = load_iris()
iris1 = pd.DataFrame(iris.data, columns=iris['feature_names'])
data = iris1[['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)']]


In [38]:
# confirm that dataframe has been read in. 
iris1

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [39]:
# create function that takes dataframe and cluster as input data.
# create mu, sigma and pi.
# pg 350, 13.3 in textbook as guide

def em_initialization(iris1,k):
   
    n, d = iris1.shape

#  generate lists
    mu_lst = []
    pi_lst = []
    for i in range(k):
        for j in range(d):
            mu = np.random.uniform(iris1[j].min(), iris1[j].max())
            mu_lt.append(mu)
        pi_lst.append(1/k)

    mu = np.array(mus_list)
    mu = mu.reshape(k, d)
    pi = np.array(pi_lst)

# d X d
    sigmas = np.array([np.eye(d)] * d)

    return mu, sigmas, pi

In [40]:
# create em_algorithm, numpy array
# iterate through expectation step
# create maximization step
# restimate pi, sigma and mu


def em_algorithm(iris1,k,mu,sigmas,pi,eps,maxi):
    iris2 = iris1.to_numpy()     
    n, d = iris2.shape    
    ll_old = 0

    for iteration in range(maxi):
        ll_new = 0

# create expectation-step
        es = np.zeros((k, n))
        for i in range(k):
            for j in range(n):
# do posterior probability
                es[i, j] = pi[i] * mvn(mu[i], sigmas[i]).pdf(iris2[j])
        es /= es.sum(0)

# create maximization-step
        
        mu = np.zeros((k, d))
        for i in range(k):
            for j in range(n):
                mu[i] += es[i, j] * data[j]
            mu[i] /= es[i, :].sum()

#   re-estimate sigmas like in 13.3 of text
        sigmas = np.zeros((k, d, d))
        for i in range(k):
            for j in range(n):
                ys = np.reshape(iris2[j] - mu[i], (d, 1))
                sigmas[i] += es[i, j] * np.dot(ys, ys.T)
            sigmas[i] /= es[i, :].sum()

# re-estimate priors
        pi = np.zeros(k)
        for i in range(k):
            for j in range(n):
                pi[i] += es[i, j]
        pi /= n



In [47]:
# create main function

if __name__ == '__main__':
    
    start = time.time()

    if len(sys.argv) < 4:
        print('arguments passed')
        #sys.exit()

print('Filename: ', sys.argv[1])
print('Cluster Value: ', sys.argv[2])
    
# accept inputs
   # try:
#k = int(sys.argv[2])
   # except ValueError:
print('Enter a numeric integer')
        #sys.exit()

   # try:
# eps = float(sys.argv[3])
    #except ValueError:
print('Enter a float value')
        #sys.exit()
        
          
iris1 = read('iris.txt')
iris2 = iris1.iloc[:, 0:4]  

# initialize
mu, sigmas, pi = em_init(iris2,k)

# begin EM algorithm
maxi = 10000  
mu, sigmas, pi, iteration, es2 = em_algorithm(iris2, k, mu, sigmas, pi, eps, maxi)

for c, mu in enumerate(mu):
    print(f'\n The mean for cluster {c} is {mu}')

for c, si in enumerate(sigmas):
    print(f'\n The covariance matrix for the cluster {c} is:\n{si}')


    iris3 = pd.DataFrame(es2)
    maxValueIndex = iris3.idxmax(axis=0)
    print('The final cluster is:\n', maxValueIndex)
    iris3 = pd.DataFrame(maxValueIndex)

    unique_elements, counts_elements = np.unique(maxValueIndex, return_counts=True)
    iris4 = pd.DataFrame(counts_elements)
    print('The size of each cluster is:\n', iris4)

    # concatenate cluster with original datarame
    iris5 = pd.concat([iris1, iris3], axis=1, ignore_index=True)
    iris5.columns = ['a1', 'a2', 'a3', 'a4', 'type', 'cluster']   
    iris5 = iris5.assign(New=1)     
    iris5 = iris5.groupby(['type', 'cluster'], as_index=False)['New'].sum()

    # find max count from confusion matrix
    cf_1 = iris5[iris5.type == 'Iris-setosa'].New.max()
    cf_2 = iris5[iris5.type == 'Iris-versicolor'].New.max()
    cf_3 = iris5[iris5.type == 'Iris-virginica'].New.max()
    purity = (cf_1+cf_2+cf_3) / iris5.New.sum()
    print('\n The purity is:{:.2f}'.format(purity))

arguments passed
Filename:  -f
Cluster Value:  C:\Users\brigg\AppData\Roaming\jupyter\runtime\kernel-ac0e009c-00d3-44a9-b835-f7dc7394e541.json
Enter a numeric integer
Enter a float value


NameError: name 'read' is not defined